In [1]:
# 데이터 불러오기
import pandas as pd
dataset = pd.read_csv('./total_price.csv')

# Del Unnamed: 0
del dataset['Unnamed: 0']

# l1~l4 line Setting
for step in ['l1', 'l2', 'l3', 'l4'] : 
    dataset[step] = dataset[step].apply(lambda x : str(x)).apply(lambda x : x[1:]).apply(lambda x : float(x))
    
# Warning Message
import warnings
warnings.filterwarnings(action='ignore') 

# 작업하는 모든 코드 리스트
code_list = dataset.code.unique()

In [2]:
# 오늘의 예측 데이터프레임
today_df = pd.DataFrame()

# 진행중 계산
step = 1

# 모든 코드 진행
for code in code_list[:1] : 
    
    # 코드 변환
    code_1 = code
    code_2 = str(code).rjust(6, '0')
    
    # 진행 결과문 표시
    if step % 250 == 1 :
        print('현재 {:4}번째 종목 진행중...({:5.2f}%)' .format(step, 100*step/len(code_list)))
    step += 1
    
    # 코드에 해당하는 데이터만 추출
    data = dataset[dataset['code'] == code_1]
    
    # 내일 종가 예측
    data['y'] = data['close'].shift(-1)
    data = data.dropna()
    
    # X,y 
    X = data[['open', 'high', 'low', 'close', 'trading_volume', 'score', 'index',
              'probability', 'l1', 'l2', 'l3', 'lgap', 'lrate']]
    y = data['y']
    
    # Model
    import tensorflow as tf
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(units=8, activation='tanh', input_shape=(X.shape[-1],)),
            tf.keras.layers.Dense(units=8, activation='tanh'),
            tf.keras.layers.Dense(units=8, activation='tanh'),
            tf.keras.layers.Dense(units=8, activation='tanh'),
            tf.keras.layers.Dense(units=8, activation='tanh'),
            tf.keras.layers.Dense(units=8, activation='tanh'),
            tf.keras.layers.Dense(units=1)])
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.07), loss='mse')
    history = model.fit(X, y, epochs=30, batch_size=128, validation_split=0.25)
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')]

현재    1번째 종목 진행중...( 0.05%)
Train on 675 samples, validate on 226 samples
Epoch 1/30
675/675 [==============================] - 0s 676us/sample - loss: 28474976.2131 - val_loss: 1809567.3518
Epoch 2/30
675/675 [==============================] - 0s 24us/sample - loss: 1548789.4846 - val_loss: 1832186.2146
Epoch 3/30
675/675 [==============================] - 0s 26us/sample - loss: 1539370.5154 - val_loss: 1460001.6460
Epoch 4/30
675/675 [==============================] - 0s 26us/sample - loss: 1530918.8331 - val_loss: 1222873.5708
Epoch 5/30
675/675 [==============================] - 0s 26us/sample - loss: 1542873.5298 - val_loss: 1397867.9004
Epoch 6/30
675/675 [==============================] - 0s 24us/sample - loss: 1551658.0796 - val_loss: 2945699.6195
Epoch 7/30
675/675 [==============================] - 0s 25us/sample - loss: 1616641.2369 - val_loss: 2397722.6106
Epoch 8/30
675/675 [==============================] - 0s 24us/sample - loss: 1566565.7565 - val_loss: 1586119.3075
Epoc

In [3]:
model.evaluate(X,y)

901/901 [==============================] - 0s 27us/sample - loss: 1502233.1761


1502233.176054384